In [ ]:
# !pip install bnlp_toolkit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls
!pwd

drive  sample_data
/content


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import pickle
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from  keras . utils  import  to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D,Conv2D, MaxPooling1D, Embedding, Flatten
from keras.models import Model
from keras.initializers import Constant
from keras.layers import Embedding

from pandas import DataFrame
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/WSD Dataset .csv')
# stop_words = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/Stopwords.csv')

In [ ]:
# df1 = pd.DataFrame(df[0:539])

In [ ]:
df.head()

,ambiguous_word,meaning,Text,Unnamed: 3
0,মাথা,অঙ্গ বিশেষ,মাথা হলো মানবদেহে বিদ্যমান সবচেয়ে ভারি অংশ।,NaN
1,মাথা,অঙ্গ বিশেষ,"মাথা অঙ্গের একটি অংশ যাতে সাধারণত চোখ, কান, না...",NaN
2,মাথা,অঙ্গ বিশেষ,মাথার উপর ভারি ডিসি পাথরটা ছেড়ে দিলেন তিনি ।,NaN
3,মাথা,অঙ্গ বিশেষ,মাথা বেল করে দেওয়া উচিত ছিল।,NaN
4,মাথা,অঙ্গ বিশেষ,মাথার উপরের এবং সামনের অংশ,NaN


In [ ]:
len(set(df['meaning']))

465

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9486 entries, 0 to 9485
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ambiguous_word  9486 non-null   object
 1   meaning         9486 non-null   object
 2   Text            9486 non-null   object
 3   Unnamed: 3      2 non-null      object
dtypes: object(4)
memory usage: 296.6+ KB


In [ ]:
df = df.drop(['Unnamed: 3'] , axis=1)


In [ ]:
df.head()

,ambiguous_word,meaning,Text
0,মাথা,অঙ্গ বিশেষ,মাথা হলো মানবদেহে বিদ্যমান সবচেয়ে ভারি অংশ।
1,মাথা,অঙ্গ বিশেষ,"মাথা অঙ্গের একটি অংশ যাতে সাধারণত চোখ, কান, না..."
2,মাথা,অঙ্গ বিশেষ,মাথার উপর ভারি ডিসি পাথরটা ছেড়ে দিলেন তিনি ।
3,মাথা,অঙ্গ বিশেষ,মাথা বেল করে দেওয়া উচিত ছিল।
4,মাথা,অঙ্গ বিশেষ,মাথার উপরের এবং সামনের অংশ


In [ ]:
texts = list(df['Text'])

In [ ]:
label = list(df['meaning'])

In [ ]:
macronum=sorted(set(df['meaning']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

df['meaning']=df['meaning'].apply(fun)

In [ ]:
label = list(df['meaning'])

In [ ]:
# max(label)

# Total no. of class 529 (sense) 

In [ ]:
def clean_punct(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#|।|’|‘]', r'', sentence)
    cleaned1 = re.sub(r'[.|,|(|)|\|/]', r'', cleaned)
  #  cleaned = re.sub(r'[০|১|২|৩|৪|৫|৬|৭|৮|৯]', r'', cleaned1)
    cleaned1 = re.sub(r'[-|=]', r' ', cleaned1)
    return cleaned1

In [ ]:
# stop_words
# set_stop = set(stop_words['words'])

In [ ]:
# len(set_stop)

In [ ]:
def pre_process(data):
    i=0
    str1=' '
    final_string = []
    final_words = []
    all_negative_words = []
    s=''

    for sentence in data:
        filtered_sentence = []
        for w in str(sentence).split():
            for cleaned_word in clean_punct(w).split():
                if len(cleaned_word)>2:
                   # print(cleaned_word)
                    # if((cleaned_word) not in set_stop):
                        # s = stemmer.stem_word(cleaned_word)
                        s = cleaned_word
                        #print(s)
                        if len(s)>2:
                            final_words.append(s)
                            filtered_sentence.append(s)
                    
                else:
                    continue

        str1 = " ".join(filtered_sentence)
        final_string.append(str1)
    return final_string

In [ ]:
import re
re.compile('<title>(.*)</title>')
texts = pre_process(texts)

In [ ]:
texts

In [ ]:
total_texts = pre_process(texts)

In [ ]:
# append the ambiguous word at last
# for i in range(len(ambi_word)):
#     ambi_word[i] = str(ambi_word[i])

# for i in range(len(texts)):
#   texts[i] = texts[i] + " " + df1['ambiguous_word'][i]

In [ ]:
df['text']= texts

In [ ]:
df.drop(['Text','ambiguous_word'],axis=1,inplace=True)

In [ ]:
df.head()

,meaning,text
0,8,মাথা হলো মানবদেহে বিদ্যমান সবচেয়ে ভারি অংশ
1,8,মাথা অঙ্গের একটি অংশ যাতে সাধারণত চোখ কান নাক ...
2,8,মাথার উপর ভারি ডিসি পাথরটা ছেড়ে দিলেন তিনি
3,8,মাথা বেল করে দেওয়া উচিত ছিল
4,8,মাথার উপরের এবং সামনের অংশ


In [ ]:
def get_embedding_vectors(word_index, embedding_size=300):
    embeddingsIndex = {}
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_size))
    with open(f"/content/drive/MyDrive/Colab Notebooks/Dataset/bn_glove.300d.txt") as f:
        for line in tqdm(f, "Reading Glove"):
            values = line.split(' ')
           # print(values)
            word = values[0]
            embeddingVector = np.array([float(val) for val in values[1:]])
            embeddingsIndex[word] = embeddingVector
    embeddingMatrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        embeddingVector = embeddingsIndex.get(word)
        if embeddingVector is not None:
            # words not found in embedding index will be all-zeros.
            embeddingMatrix[i] = embeddingVector
    
    return embeddingMatrix 

# MAx Unique word

In [ ]:
words = []
count = 0
for line in total_texts:
  sen = line.split()
  for word in sen:
    # print(word)
    if word not in words:
      words.append(word)
      count += 1
print(count)

15007


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = 1761)
tokenizer.fit_on_texts(total_texts)
sequences = tokenizer.texts_to_sequences(total_texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 14964 unique tokens.


In [ ]:
word_index

In [ ]:
embedding_matrix = get_embedding_vectors(word_index)

Reading Glove: 178153it [00:26, 6819.25it/s]


In [ ]:
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.829841, -0.452853,  0.124189, ...,  0.659246,  0.247762,
         0.930659],
       [-0.13826 ,  0.290732, -0.026134, ...,  0.657288,  0.12792 ,
        -0.051071],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.202973,  0.045013,  0.039   , ...,  0.12112 , -0.683351,
         0.106692],
       [ 0.179712,  0.236013,  0.09481 , ..., -0.443959,  0.155647,
         0.502211]])

In [ ]:
# len(data['tokenizer'].word_index)

In [ ]:
d = set()

for s in total_texts:
    for ss in list(s.split()):
        d.add(ss)

In [ ]:
len(d)

15007

In [ ]:
def my_max(x , y):
  if x > y:
    return x
  return y 



# Maximum sequence length 129

In [ ]:
max_sequence_length = 0
for text in total_texts:
  max_sequence_length = my_max(max_sequence_length , len(text))


In [ ]:
print(max_sequence_length)

151


# Total unique words 13836

In [ ]:
def load_data(num_words, sequence_length, test_size=0.3, oov_token=None):
    tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
    num_classes=465
    y = to_categorical(np.asarray(label), num_classes)
   
   
   
    tokenizer.fit_on_texts(total_texts)

    X = tokenizer.texts_to_sequences(total_texts)
    X = np.array(X)
    X = pad_sequences(X, maxlen=sequence_length)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1)
    data = {}
    data["X_train"] = X_train
    data["X_test"]= X_test
    data["y_train"] = y_train
    data["y_test"] = y_test
    data["tokenizer"] = tokenizer
    return data

In [ ]:
MAX_SEQUENCE_LENGTH = max_sequence_length    #max([len(s.split()) for s in texts]) 
MAX_NUM_WORDS = 15007
vocab_size = MAX_NUM_WORDS
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.3

data = load_data(MAX_NUM_WORDS , MAX_SEQUENCE_LENGTH,VALIDATION_SPLIT)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU 
from keras.layers import GRU, LSTM, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [ ]:
# class Attention(Layer):
#     """
#     Keras Layer that implements an Attention mechanism for temporal data.
#     Supports Masking.
#     Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
#     # Input shape
#         3D tensor with shape: `(samples, steps, features)`.
#     # Output shape
#         2D tensor with shape: `(samples, features)`.
#     :param kwargs:
#     Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
#     The dimensions are inferred based on the output shape of the RNN.
#     Example:
#         model.add(LSTM(64, return_sequences=True))
#         model.add(Attention())
#     """   
   
   
#     def __init__(self, step_dim,
#                  W_regularizer=None, b_regularizer=None,
#                  W_constraint=None, b_constraint=None,
#                  bias=True, **kwargs):
#         self.supports_masking = True
#         self.init = initializers.get('glorot_uniform')

#         self.W_regularizer = regularizers.get(W_regularizer)
#         self.b_regularizer = regularizers.get(b_regularizer)

#         self.W_constraint = constraints.get(W_constraint)
#         self.b_constraint = constraints.get(b_constraint)

#         self.bias = bias
#         self.step_dim = step_dim
#         self.features_dim = 0
#         super(Attention, self).__init__(**kwargs)          
            
            
#         self.features_dim = 0
#         super(Attention, self).__init__(**kwargs)

#     def build(self, input_shape):
#         assert len(input_shape) == 3
#         shape=(input_shape[-1],)
#         self.W = self.add_weight((shape),
#                                  initializer=self.init,
#                                  name='{}_W'.format(self.name),
#                                  regularizer=self.W_regularizer,
#                                  constraint=self.W_constraint)
#         self.features_dim = shape

#         if self.bias:
#             self.b = self.add_weight((shape),
#                                      initializer='zero',
#                                      name='{}_b'.format(self.name),
#                                      regularizer=self.b_regularizer,
#                                      constraint=self.b_constraint)
#         else:
#             self.b = None
#         self.built = True
    
#     def compute_mask(self, input, input_mask=None):
#         return None

#     def call(self, x, mask=None):
#         features_dim = self.features_dim
#         step_dim = self.step_dim
#         eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
#                         K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
#         if self.bias:
#             eij += self.b
#         eij = K.tanh(eij)
#         a = K.exp(eij)
#         if mask is not None:
#             a *= K.cast(mask, K.floatx())
#         a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
#         a = K.expand_dims(a)
#         weighted_input = x * a
#         return K.sum(weighted_input, axis=1)

#     def compute_output_shape(self, input_shape):
#         return input_shape[0],  self.features_dim

In [ ]:
# #max_features = 30000
# max_features = 14964 + 1
# max_len = 151
# def build_model(max_len, max_features, EMBEDDING_DIM, embedding_matrix):
#     input_words = Input((max_len, ))
#     x_words = Embedding(max_features,
#                         EMBEDDING_DIM,
#                         weights=[embedding_matrix],
#                         mask_zero=True,
#                         trainable=False)(input_words)
#     x_words = SpatialDropout1D(0.2)(x_words)
#     x_words = Bidirectional(LSTM(128, return_sequences=True))(x_words)
#     x_words = Bidirectional(LSTM(128, return_sequences=True))(x_words)
    
#     x = Attention(max_len)(x_words)
#     #x = GlobalMaxPooling1D()(x)
#     #x = GlobalAveragePooling1D()(x)
#     x = Dropout(0.2)(x)
#     x = Dense(64, activation='relu')(x)
#     x = Dropout(0.2)(x)
#     pred = Dense(16, activation='softmax')(x)

#     model = Model(inputs=input_words, outputs=pred)
#     return model

# model = build_model(max_len, max_features, EMBEDDING_DIM, embedding_matrix)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()
# '''
# MAX_SEQUENCE_LENGTH = 11276    #max([len(s.split()) for s in texts]) 
# MAX_NUM_WORDS = 109803 + 1
# vocab_size = MAX_NUM_WORDS
# EMBEDDING_DIM = 300
# VALIDATION_SPLIT = 0.2

# data = load_data(MAX_NUM_WORDS , MAX_SEQUENCE_LENGTH,VALIDATION_SPLIT)
# '''

In [ ]:
# embedding_vecor_length = 300
# MAX_NUM_WORDS = 1761
# model = Sequential()
# model.add(Embedding(MAX_NUM_WORDS, embedding_vecor_length,  weights=[embedding_matrix], input_length=max_sequence_length))
# model.add(LSTM(300))

# model.add(Dense(27, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())

In [ ]:
embedding_vecor_length = 300
MAX_NUM_WORDS = 14965
model = Sequential()
model.add(Embedding(MAX_NUM_WORDS, embedding_vecor_length,  weights=[embedding_matrix], input_length=max_sequence_length))
model.add(Bidirectional(LSTM(300)))

model.add(Dense(465, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 151, 300)          4489500   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 600)               1442400   
_________________________________________________________________
dense (Dense)                (None, 465)               279465    
Total params: 6,211,365
Trainable params: 6,211,365
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(data['X_train'], data['y_train'], validation_data=(data['X_test'], data['y_test']),
                    epochs=100,
                    verbose=2)

Epoch 1/100
208/208 - 68s - loss: 5.0461 - accuracy: 0.1277 - val_loss: 3.7308 - val_accuracy: 0.2713
Epoch 2/100
208/208 - 32s - loss: 2.4161 - accuracy: 0.5086 - val_loss: 2.2233 - val_accuracy: 0.5330
Epoch 3/100
208/208 - 32s - loss: 1.0571 - accuracy: 0.8041 - val_loss: 1.7081 - val_accuracy: 0.6244
Epoch 4/100
208/208 - 32s - loss: 0.4402 - accuracy: 0.9358 - val_loss: 1.5078 - val_accuracy: 0.6634
Epoch 5/100
208/208 - 32s - loss: 0.1789 - accuracy: 0.9824 - val_loss: 1.4007 - val_accuracy: 0.6880
Epoch 6/100
208/208 - 32s - loss: 0.0879 - accuracy: 0.9908 - val_loss: 1.3780 - val_accuracy: 0.6911
Epoch 7/100
208/208 - 31s - loss: 0.0502 - accuracy: 0.9946 - val_loss: 1.3909 - val_accuracy: 0.6845
Epoch 8/100
208/208 - 31s - loss: 0.0368 - accuracy: 0.9955 - val_loss: 1.3410 - val_accuracy: 0.6985
Epoch 9/100
208/208 - 32s - loss: 0.0294 - accuracy: 0.9958 - val_loss: 1.3516 - val_accuracy: 0.7020
Epoch 10/100
208/208 - 32s - loss: 0.0238 - accuracy: 0.9965 - val_loss: 1.3303 - 

In [ ]:
model_eval = model.evaluate(data['X_test'], data['y_test'])

In [ ]:
data['X_train'].shape

In [ ]:
data['X_test'].shape

In [ ]:
data['y_train'].shape

In [ ]:
data['y_test'].shape

In [ ]:
from keras.layers import TimeDistributed

In [ ]:
def get_sequence(n_timesteps):
	# create a sequence of random numbers in [0,1]
	X = array([random() for _ in range(n_timesteps)])
	# calculate cut-off value to change class values
	limit = n_timesteps/4.0
	# determine the class outcome for each item in cumulative sequence
	y = array([0 if x < limit else 1 for x in cumsum(X)])
	# reshape input and output data to be suitable for LSTMs
	X = X.reshape(1, n_timesteps, 1)
	y = y.reshape(1, n_timesteps, 1)
	return X, y

In [ ]:
def get_lstm_model(n_timesteps, backwards):
	model = Sequential()
	model.add(LSTM(20, input_shape=(n_timesteps, 1), return_sequences=True, go_backwards=backwards))
	model.add(TimeDistributed(Dense(1, activation='softmax')))
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	return model
 
def get_bi_lstm_model(n_timesteps, mode):
	model = Sequential()
	model.add(Bidirectional(LSTM(20, return_sequences=True), input_shape=(n_timesteps, 1), merge_mode=mode))
	model.add(TimeDistributed(Dense(1, activation='softmax')))
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	return model
 
def train_model(model, n_timesteps):
	loss = list()
	for _ in range(250):
		# generate new random sequence
		X,y = get_sequence(n_timesteps)
		# fit model for one epoch on this sequence
		hist = model.fit(X, y, epochs=1, batch_size=1, verbose=0)
		loss.append(hist.history['loss'][0])
	return loss
 
 
n_timesteps = 10
results = DataFrame()
# lstm forwards
model = get_lstm_model(n_timesteps, False)
results['lstm_forw'] = train_model(model, n_timesteps)
# lstm backwards
model = get_lstm_model(n_timesteps, True)
results['lstm_back'] = train_model(model, n_timesteps)
# bidirectional concat
model = get_bi_lstm_model(n_timesteps, 'concat')
results['bilstm_con'] = train_model(model, n_timesteps)
# line plot of results
results.plot()
pyplot.show()

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
def confussion_mat(model):
    # Predict the values from the validation dataset
    Y_pred = model.predict(data['X_test'])
    # Convert predictions classes to one hot vectors 
    Y_pred_classes = np.argmax(Y_pred,axis = 1) 
    # Convert validation observations to one hot vectors
    Y_true = np.argmax(data['y_test'],axis = 1) 
    # compute the confusion matrix
    confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
    # plot the confusion matrix
    f,ax = plt.subplots(figsize=(15, 15))
    sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.show()

In [ ]:
def history2(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train','validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','validation'], loc='upper left')
    plt.show()

In [ ]:
history2(history)

In [ ]:
#extract labels from encoder
labels = list(df['meaning'])

In [ ]:
confussion_mat(model)

# BERT

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertModel, BertTokenizer, pipeline

In [ ]:
!pip install git+https://github.com/sagorbrur/bangla-bert.git

In [ ]:
!ls

In [ ]:
model_name = "sagorsarker/bangla-bert-base"
# model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

In [ ]:
masked_text = []
for tmp in texts:
    masked_text.append("[CLS] " + tmp + " [SEP]")


def get_embedding_matrix(masked_text):
    inputs = tokenizer(masked_text, return_tensors="pt", padding=True, truncation = True,  max_length =129)
    outputs = model(**inputs)
    #last_hidden_states = outputs.last_hidden_state 


    return outputs[1]

In [ ]:
# v = get_embedding_matrix(masked_text)

In [ ]:
a = []
while(1):
    a.append(1)